This first cell does setup. It takes data by reading `uni_data.csv` into a Pandas dataframe, but if you combine this with `scrap_wiki.ipynb` you could probably just read it directly.

In [41]:
# Run `pip install ultimate_sitemap_parser` first
from usp.tree import sitemap_tree_for_homepage
from pprint import pprint
import pandas as pd

In [14]:
def getSitemap(dfRow):
    return sitemap_tree_for_homepage(dfRow['url'])

In [15]:
# Load university dataframe
data = pd.read_csv('uni_data.csv')

In [16]:
# Apply getSitemap() to every row of data
# getSitemap() returns a sitemap tree object, which is stored in a new column
data['sitemap tree'] = data.apply(getSitemap, axis=1)

2020-06-27 12:43:40,571 WARNING usp.tree [15023/MainThread]: Assuming that the homepage of https://utoronto.ca is https://utoronto.ca/
2020-06-27 12:43:40,571 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://utoronto.ca/robots.txt...
2020-06-27 12:43:40,572 INFO usp.helpers [15023/MainThread]: Fetching URL https://utoronto.ca/robots.txt...
2020-06-27 12:43:40,876 INFO usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL https://utoronto.ca/robots.txt...
2020-06-27 12:43:40,878 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://utoronto.ca/sitemap_index.xml.gz...
2020-06-27 12:43:40,880 INFO usp.helpers [15023/MainThread]: Fetching URL https://utoronto.ca/sitemap_index.xml.gz...
2020-06-27 12:43:41,369 WARNING usp.helpers [15023/MainThread]: Request for URL https://utoronto.ca/sitemap_index.xml.gz failed: 404 Not Found
2020-06-27 12:43:41,370 INFO usp.helpers [15023/MainThread]: Not retrying for URL https://utoronto.ca/si

2020-06-27 12:43:50,027 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://www.york.ac.uk/sitemap/sitemap-index.xml...
2020-06-27 12:43:50,029 INFO usp.helpers [15023/MainThread]: Fetching URL https://www.york.ac.uk/sitemap/sitemap-index.xml...
2020-06-27 12:43:50,265 WARNING usp.helpers [15023/MainThread]: Request for URL https://www.york.ac.uk/sitemap/sitemap-index.xml failed: 404 Not Found
2020-06-27 12:43:50,268 INFO usp.helpers [15023/MainThread]: Not retrying for URL https://www.york.ac.uk/sitemap/sitemap-index.xml
2020-06-27 12:43:50,270 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from https://www.york.ac.uk/sitemap-index.xml.gz...
2020-06-27 12:43:50,272 INFO usp.helpers [15023/MainThread]: Fetching URL https://www.york.ac.uk/sitemap-index.xml.gz...
2020-06-27 12:43:50,534 WARNING usp.helpers [15023/MainThread]: Request for URL https://www.york.ac.uk/sitemap-index.xml.gz failed: 404 Not Found
2020-06-27 12:43:50,537 INFO usp.hel

2020-06-27 12:43:54,652 INFO usp.helpers [15023/MainThread]: Fetching URL http://harvard.edu/.sitemap.xml...
2020-06-27 12:43:54,912 WARNING usp.helpers [15023/MainThread]: Request for URL http://harvard.edu/.sitemap.xml failed: 404 Not Found
2020-06-27 12:43:54,915 INFO usp.helpers [15023/MainThread]: Not retrying for URL http://harvard.edu/.sitemap.xml
2020-06-27 12:43:54,917 INFO usp.fetch_parse [15023/MainThread]: Fetching level 0 sitemap from http://harvard.edu/sitemap...
2020-06-27 12:43:54,918 INFO usp.helpers [15023/MainThread]: Fetching URL http://harvard.edu/sitemap...
2020-06-27 12:43:55,180 INFO usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL http://harvard.edu/sitemap...
2020-06-27 12:43:55,182 ERROR usp.fetch_parse [15023/MainThread]: Parsing sitemap from URL http://harvard.edu/sitemap failed: Unsupported root element 'html'.
2020-06-27 12:43:55,184 WARNING usp.tree [15023/MainThread]: Assuming that the homepage of http://yale.edu is http://yale.edu/
2020-06-

In [45]:
data

,name,locality,state,country,url,sitemap tree
0,University of Toronto,Toronto,Ontario,Canada,https://utoronto.ca,"IndexWebsiteSitemap(url=https://utoronto.ca/, ..."
1,University of York,"Heslington, York",NaN,England,https://www.york.ac.uk/,IndexWebsiteSitemap(url=https://www.york.ac.uk...
2,Harvard University,Cambridge,Massachusetts,United States,http://harvard.edu,"IndexWebsiteSitemap(url=http://harvard.edu/, s..."
3,Yale University,New Haven,Connecticut,United States,http://yale.edu,"IndexWebsiteSitemap(url=http://yale.edu/, sub_..."


In [55]:
# Search the URL for a given search term
# If we need to implement regex here (and I hope to God we don't) I can implement it
def urlSearch(dfRow, searchTerm):
    sitemap = dfRow['sitemap tree']
    searchResults = [page.url for page in sitemap.all_pages() if searchTerm in page.url]
    return searchResults

# Encapsulate urlSearch() in a lambda so that it has only one parameter exposed
# to do a specific search
facSearch = lambda dfRow: urlSearch(dfRow, 'faculty')

# Search each row of the dataframe and append results to new column
data['search results'] = data.apply(facSearch, axis=1)

# @Emily: This may be a really dumb idea, but could we just choose the result with
# the shortest length???

In [62]:
data[['name', 'search results']]

,name,search results
0,University of Toronto,"[https://www.utoronto.ca/faculty-staff, https:..."
1,University of York,[]
2,Harvard University,[https://www.harvard.edu/president/speech/2005...
3,Yale University,[]


In [61]:
# To show you how many URLs we need to sort through
data.loc[0]['search results']

['https://www.utoronto.ca/faculty-staff',
 'https://www.utoronto.ca/news/uoftbts16-orientation-faculty-too',
 'https://www.utoronto.ca/news/new-look-u-t-s-faculty-information',
 'https://www.utoronto.ca/news/carding-controversy-u-t-faculty-members-speak-out',
 'https://www.utoronto.ca/news/u-t-s-faculty-medicine-forging-new-paths-global-women-s-health',
 'https://www.utoronto.ca/news/meet-u-t-scarborough-s-new-vice-dean-faculty-affairs-equity',
 'https://www.utoronto.ca/news/first-indigenous-faculty-scholar-u-t-s-jackman-humanities-institute-connects-community-through',
 'https://www.utoronto.ca/news/university-toronto-faculty-experts-and-students-respond-trump-s-travel-ban',
 'https://www.utoronto.ca/news/testing-continues-faculty-medicine-prep-lab-retrofits-gets-underway',
 'https://www.utoronto.ca/news/tots-and-profs-u-t-staff-faculty-bring-their-kids-work',
 'https://www.utoronto.ca/news/toronto-tuktoyaktuk-u-t-faculty-students-take-science-rendezvous-across-canada',
 'https://www.